## Convert Classic Esri Story Map Series

**Welcome!**  
This notebook will guide you through converting a Classic Esri Story Map Series to an ArcGIS StoryMap Collection.

**How to run this notebook**  
 - Click on the text "Setup and authenticate" below. 
 - There are two types of cells, Markdown (formatted notes) and Code.
 - An indicator -- typically a vertical blue line -- should highlight that you have selected the "Setup and authenticate" Markdown cell. 
 - Once selected, click the "Play" button in the toolbar above to run the cell and advance to the next Code cell.
 - Click the "Play" button a second time to run the code cell.
 - After several seconds a "Setup Notebook" button should appear. Click the button to begin setup and authentication.
 - After each cell completes, click the text within the following Markdown cell.
 - Click the "Play" button to advance to the Code cell, then click the "Play" button a second time to make a button appear.
 - Click the button to run the code in the cell. 

 **Notes**  
 - Some code cells may take a while to execute. You can monitor the status by viewing the small circle in the top right of the page.
 - If you click on a code cell it will expand showing you the behind-the-scenes Python code. 
 - For a cleaner interface select View > Collapse All Code in the menu bar above to hide the code .
 - If at any point you get stuck and want to start over, just click Kernel > Restart Kernel and Clear Outputs of All Cells... in the menu bar 

**tldr;**

In [ ]:
# Run this cell to display Notebook details
from IPython.display import display, Markdown

# Display details of what this notebook does
tldr_md = """
**What this notebook does**  
- Fetch JSON from an ArcGIS Online hosted Classic Esri Story Map Series App 
- Convert each tab/bullet/accordion into its own ArcGIS StoryMap with the cover supressed
- Uses the BeautifulSoup library to parse description.html from the side panel text and convert to StoryMap objects
- Identifies the main stage media type and converts each to the corresponding StoryMap object 
- Once converted, each ArcGIS StoryMap will need to be opened in a browser tab in order to complete the Story Checker 
- Once all are published, an ArcGIS StoryMap Collection is created that contains the converted app to replicate the classic app look and feel
- Note: Any entries that were hidden in the classic app will be published and will be visible by default. If it is desired that they not appear they can be removed from the Collection after publishing 
- Also, as there is no AGSM Collection equivalent to the accordion layout, these layouts will be converted to the Tabbed format.
"""
display(Markdown(tldr_md))

TO DO - create conversion tool for classic swipe (second tab in Katrina story)

## 1. Setup and authenticate

In [ ]:
# In the toolbar above, select View > Collapse All Code to hide the code.
print("Initializing...")
# Cell 1. Import packages, config, AGO authentication and helper functions
import sys, json
# Set the path to access the helper functions
sys.path.append('/arcgis/home')

from storymap_series_conversion_utils import (
    detect_environment,
    authenticate_gis,
    initialize_ui,
    fetch_story_data_btn,
    extract_and_display_settings_btn,
    process_entries_btn,
    create_storymaps_btn,
    create_collection_btn,
    check_folder_btn,
    create_folder_btn,
    move_items_to_folder_btn,
)
import arcgis
import pandas as pd
from functools import partial
import ipywidgets as widgets
import warnings
from bs4 import MarkupResemblesLocatorWarning

# Suppress the BeautifulSoup warning when ArcGIS Notebooks thinks HTML looks like a locator/filename
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Set Pandas dataframe display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',1000)

# Define a context dictionary to manage state explicity
context = {
    "gis": None,
    "classic_id": None,
    "classic_item": None,
    "classic_item_data": None,
    "classic_story_title": None,
    "classic_story_subtitle": None,
    "classic_story_type": None,
    "classic_story_panel_position": None,
    "classic_story_theme": None,
    "new_theme": None,
    "default_thumbnail_path": "",
    "entries": [],
    "entry_titles": [],
    "html_elements": [],
    "content_nodes": [],
    "main_stage_contents": [],
    "invalid_webmaps": [],
    "published_storymap_items": [],
    "thumbnail_paths": [],
    "collection_title": None,
    "collection_url": None,
    "collection_id": None,
    "folder_name": None,
}
# Detect the current environment
current_env, env_string = detect_environment()
print(f"Currently running in {env_string}")

output1 = initialize_ui("output")
def setup_notebook_btn(button):
    with output1:
        print("Setting up the notebook environment...")
        # Print Python and ArcGIS for Python versions
        # since things can change between versions
        print(f"\tPython version: ",sys.version)
        print(f"\tArcGIS for Python API / StoryMap module version: ",arcgis.__version__)
        # Connect to ArcGIS Online
        # Define the GIS
        authenticate_gis(context=context)
        
# Set default thumbnail path
default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"
context['default_thumbnail_path'] = default_thumbnail_path

# Collect map extents for troubleshooting thumbnail generation
# map_extents = []
# webmap_jsons = []

# Create widgets
btn1 = initialize_ui("button", description="Setup Notebook", width="200px")
output1 = initialize_ui("output")
# Display widgets
display(btn1)
btn1.on_click(setup_notebook_btn)
display(output1)

## 2. Fetch the Data from the Classic Story Map
This cell fetches the classic StoryMap item and parses its JSON data.

In [ ]:
# Cell 2: Input the classic StoryMap ID
output2 = initialize_ui("output") 

txt2 = initialize_ui("label", value="Paste 32-digit Classic Esri Story Map id -->", width="300px")
input2 = initialize_ui("text", value="", description="Item ID:", width="400px") 
# Test values: d1799fc84e244c2f9af0e24ced4c95e1 (Bulleted) ; 597d573e58514bdbbeb53ba2179d2359 (Tabbed)
hbox2 = initialize_ui("hbox", elements=[txt2, input2]) # TO DO add error checking logic and warning if item is missing or input is incorrect
btn2 = initialize_ui("button", description="Fetch Story Data", width="200px")
display(widgets.VBox([hbox2, btn2, output2]))
btn2.on_click(partial(fetch_story_data_btn, output2=output2, input2=input2, context=context))

## 3. Parse Settings, Theme and Data

This cell extracts the theme, title, subtitle, and entries from the classic StoryMap data.

In [ ]:
# Cell 3: Extract settings and entries
output3 = initialize_ui("output") 
        
btn3 = initialize_ui("button", description="Extract Settings")
display(widgets.VBox([btn3, output3]))
btn3.on_click(partial(extract_and_display_settings_btn, output3=output3, context=context))

## 4. Loop Through and Process Each Entry's Data

In [ ]:
# Cell 4: Loop through entries to process media, content and thumbnails
output4 = initialize_ui("output")

btn4 = initialize_ui("button", description="Process Entries")
display(widgets.VBox([btn4, output4]))
btn4.on_click(partial(process_entries_btn, output4=output4, context=context))

## 5. Build an ArcGIS StoryMap with a Suppressed Cover Page for Each Entry

In [ ]:
# Cell 5: Loop through each entry and create a StoryMap for each
output5 = initialize_ui("output")

btn5= initialize_ui("button", description="Create ArcGIS StoryMaps from each entry", width='300px', height='40px')
display(widgets.VBox([btn5, output5]))
btn5.on_click(partial(create_storymaps_btn, output5=output5, context=context))

## 6. Build a Collection from the Published StoryMaps

In [ ]:
# Cell 6. Run the function to create the Collection
output6 = initialize_ui("output")

btn6 = initialize_ui("button", description="Create Collection", width='150px', height='40px')
display(widgets.VBox([btn6, output6]))
btn6.on_click(partial(create_collection_btn, output6=output6, context=context))

## 7. Create a folder in My Content to store the results (optional, but recommended) 

In [ ]:
# Cell 7: Create a folder to store the results
output7 = initialize_ui("output")
input7 = initialize_ui("text", value="", description="", width='800px')
btn7 = initialize_ui("button", description="Check for folder")
btn7_1 = initialize_ui("button", description="Create folder")

display(widgets.VBox([btn7, output7]))

btn7.on_click(partial(check_folder_btn, input7=input7, output7=output7, btn7_1=btn7_1, context=context))
btn7_1.on_click(partial(create_folder_btn, input7=input7, output7=output7, context=context))

## 8. Move content into folder for easier management

In [ ]:
# Cell 8. Move all items to the newly created folder
output8 = initialize_ui("output")

btn8 = initialize_ui("button", description="Move all items to folder", width='200px', height='40px')
display(widgets.VBox([btn8, output8]))
btn8.on_click(partial(move_items_to_folder_btn, output8=output8, context=context))